In [1]:
## imports ##
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.externals import joblib
from sklearn.linear_model import LogisticRegression
from sklearn import svm, decomposition, tree
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, ExtraTreesClassifier
from sklearn import metrics
from sklearn.model_selection import learning_curve, GridSearchCV, cross_val_score
from sklearn.metrics import classification_report, confusion_matrix
import pandas as pd
import numpy as np
import warnings
from sklearn.feature_selection import RFECV
from xgboost import XGBClassifier

warnings.simplefilter("ignore")
np.random.seed(100)

C:\Users\atedi\OneDrive\Documents\SageBionetworks\synapse\lib\site-packages\sklearn\externals\joblib\__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


## Contents:

- Gait data being used is based on the PDKit mPowerV1 data, collapsed by each healthCode ids
- Features chosen are based on a feature_engineering.ipnb
- For this ML Analysis, we will split the training-test set by 20%
- Parameters will be searched through grid search (GridSearchCV) and the score is assessed using Stratified 10-Fold Validation 
- Models being used: Logistic Regression, Xtreme Gradient Boost, Sklearn Gradient Boost and Random Forests
- Feature elimination will be done on several model using recursive feature elimination CV (Sklearn RFECV package)
- Learning Curves of each model (Work in progress)
- Best model will be dump into .pkl file (Work in progress)

## Helper Functions

In [2]:
def logreg_fit(X_train, y_train):
    pipe = Pipeline(steps=[
        ('scaler', StandardScaler()),
        ('classifier', LogisticRegression(random_state = 100))
        ])
    param = [{'classifier__penalty': ['l2'], 
              'classifier__solver': [ 'newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']}, 
             {'classifier__penalty': ['l1'], 
              'classifier__solver': [ 'liblinear', 'saga']}  
            ]

    CV = GridSearchCV(estimator = pipe, param_grid = param , scoring= "roc_auc", n_jobs = 1, cv = 10)
    CV.fit(X_train, y_train)
    return CV



def xgb_fit(X_train, y_train):
    pipe = Pipeline(steps=[
        ('scaler', StandardScaler()),
        ('classifier', XGBClassifier(seed = 100, nthread = -1))
        ])
    param = {
        "classifier__learning_rate" : [0.01, 0.05, 0.1],
        "classifier__tree_method"   : ["hist", "auto"],
        "classifier__max_depth"     : [3, 6, 8],
        "classifier__gamma"         : [1],
        "classifier__subsample"     : [0.8],
        "classifier__n_estimators"  : [100]
    }
    CV = GridSearchCV(estimator = pipe, param_grid = param , scoring= "roc_auc", n_jobs = 1, cv = 10)
    CV.fit(X_train, y_train)
    return CV
    

def gradientboost_fit(X_train, y_train):
    pca = decomposition.PCA()
    pipe = Pipeline(steps=[
        ('scaler', StandardScaler()),
        ('classifier', GradientBoostingClassifier(random_state = 100, warm_start = True))
        ])
    param = {
        'classifier__learning_rate': [0.001, 0.005, 0.01, 0.05, 0.1],
        'classifier__max_depth':[1, 2, 3, 4, 5, 6],
        'classifier__loss': ["deviance", "exponential"], ## exponential will result in adaBoost
        "classifier__n_estimators"  : [100]
    }
    CV = GridSearchCV(estimator = pipe, param_grid = param , scoring= "roc_auc", n_jobs = 1, cv = 10)
    CV.fit(X_train, y_train)
    return CV

def randomforest_fit(X_train, y_train):
    pipe = Pipeline(steps=[
        ('scaler', StandardScaler()),
        ('classifier', RandomForestClassifier(random_state = 100))
        ])
    param = {
        'classifier__max_depth':[1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
        'classifier__criterion': ["gini", "entropy"],## exponential will result in adaBoost
        'classifier__max_features': ["auto", "sqrt", "log2", None], 
        'classifier__n_estimators'  : [100, 200]
    }
    CV = GridSearchCV(estimator = pipe, param_grid = param , scoring= "roc_auc", n_jobs = 1, cv = 10)
    CV.fit(X_train, y_train)
    return CV


def printPerformance(model, X_test, y_test):
    print("Mean AUC score on K-folds: {}".format(model.best_score_))
    print("Parameter Used: {}".format(model.best_params_))
    y_true, y_pred = y_test, model.predict(X_test)
    print("ROC-AUC on Test-Set: {}".format(metrics.roc_auc_score(y_true, y_pred)))
    print("log-loss: {}".format(metrics.log_loss(y_true, y_pred)))
    print("Precision: {}".format(metrics.precision_score(y_true, y_pred)))
    print("Recall: {}".format(metrics.recall_score(y_true, y_pred)))
    print("F1-Score: {}".format(metrics.f1_score(y_true, y_pred)))

## Split to Training and Testing Sets

In [3]:
walking_train = pd.read_csv("../Data/walking_data_training.csv", index_col=0)
walking_train_imputed = pd.read_csv("../Data/walking_imputed.csv", index_col=0)
walking_test_imputed = pd.read_csv("../Data/walking_imputed_test_data.csv", index_col=0)
balance_train = pd.read_csv("../Data/balance_data_training.csv", index_col=0).dropna()
balance_X_train, balance_X_test, balance_y_train, balance_y_test = train_test_split(balance_train.drop(["healthCode", "PD"], axis = 1), balance_train["PD"], test_size=0.20, random_state = 100)
walking_X_train, walking_X_test, walking_y_train, walking_y_test = train_test_split(walking_train.drop(["healthCode", "PD"], axis = 1), walking_train["PD"], test_size=0.20, random_state = 100)

## Run Model on Walking Data

In [4]:
lr_walking_model = logreg_fit(walking_X_train, walking_y_train)
rf_walking_model = randomforest_fit(walking_X_train, walking_y_train)
gb_walking_model = gradientboost_fit(walking_X_train, walking_y_train)
xgb_walking_model = xgb_fit(walking_X_train, walking_y_train)


print("\n### Gradient Boosting Walking ###")
printPerformance(gb_walking_model, walking_X_test.dropna(), walking_y_test.dropna())
print("\n### XTreme Gradient Boosting Walking ###")
printPerformance(xgb_walking_model, walking_X_test.dropna(), walking_y_test.dropna())
print("\n### Random Forest Walking ###")
printPerformance(rf_walking_model, walking_X_test.dropna(), walking_y_test.dropna())
print("\n### Logistic Regression Walking ###")
printPerformance(lr_walking_model, walking_X_test.dropna(), walking_y_test.dropna())


### Gradient Boosting Walking ###
Mean AUC score on K-folds: 0.6869747899159663
Parameter Used: {'classifier__learning_rate': 0.1, 'classifier__loss': 'deviance', 'classifier__max_depth': 6, 'classifier__n_estimators': 100}
ROC-AUC on Test-Set: 0.7619047619047619
log-loss: 7.970548367858634
Precision: 0.8
Recall: 0.6666666666666666
F1-Score: 0.7272727272727272

### XTreme Gradient Boosting Walking ###
Mean AUC score on K-folds: 0.6935107376283847
Parameter Used: {'classifier__gamma': 1, 'classifier__learning_rate': 0.1, 'classifier__max_depth': 8, 'classifier__n_estimators': 100, 'classifier__subsample': 0.8, 'classifier__tree_method': 'hist'}
ROC-AUC on Test-Set: 0.6547619047619049
log-loss: 11.51300747496307
Precision: 0.6923076923076923
Recall: 0.5
F1-Score: 0.5806451612903226

### Random Forest Walking ###
Mean AUC score on K-folds: 0.7049486461251168
Parameter Used: {'classifier__criterion': 'gini', 'classifier__max_depth': 3, 'classifier__max_features': 'auto', 'classifier__n_es

## Confusion Matrix 

In [7]:
print(metrics.classification_report(rf_walking_model.predict(walking_X_test), walking_y_test))

              precision    recall  f1-score   support

           0       0.86      0.72      0.78        25
           1       0.61      0.79      0.69        14

    accuracy                           0.74        39
   macro avg       0.73      0.75      0.74        39
weighted avg       0.77      0.74      0.75        39



In [8]:
pd.crosstab(rf_walking_model.predict(walking_X_test), walking_y_test, rownames = ["Actual"], colnames = ["Predicted"])

Predicted,0,1
Actual,,
0,18,7
1,3,11


In [16]:
## Model Feature Importances by Gini Index
pd.Series(data = rf_walking_model.best_estimator_[1].feature_importances_, index = walking_X_test.columns).nlargest(10)

max_median_freeze_index accel_walking_features_x        0.165895
max_gait_symmetry userAccel_walking_features_z          0.132659
max_median_freeze_index accel_walking_features_z        0.124324
max_count_freeze_index userAccel_walking_features_AA    0.116638
max_median_freeze_index userAccel_walking_features_x    0.100786
max_median_freeze_index accel_walking_features_AA       0.079038
max_gait_symmetry userAccel_walking_features_x          0.070303
max_count_freeze_index userAccel_walking_features_z     0.066463
max_count_freeze_index userAccel_walking_features_x     0.055631
max_median_freeze_index accel_walking_features_y        0.049956
dtype: float64

In [17]:
## Model Feature Importances
pd.Series(data = gb_walking_model.best_estimator_[1].feature_importances_, index = walking_X_test.columns).nlargest(10)

max_gait_symmetry userAccel_walking_features_z          0.176044
max_median_freeze_index accel_walking_features_z        0.156939
max_median_freeze_index accel_walking_features_x        0.152922
max_count_freeze_index userAccel_walking_features_z     0.110416
max_count_freeze_index userAccel_walking_features_AA    0.103573
max_count_freeze_index userAccel_walking_features_y     0.084524
max_median_freeze_index accel_walking_features_AA       0.064938
max_median_freeze_index accel_walking_features_y        0.053581
max_gait_symmetry userAccel_walking_features_x          0.038694
max_count_freeze_index userAccel_walking_features_x     0.034383
dtype: float64